In [33]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import logging
from tqdm import tqdm

# 配置日志记录，便于调试和错误追踪
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

class MultiSampleTileExprDataset(Dataset):
    """
    数据集同时加载图像和表达量数据，并解析文件名中的坐标信息。

    数据目录组织要求：
      - 根目录（root_dir）：/cwStorage/nodecw_group/jijh/hest_output
      - 子文件夹命名方式：
          * {sample_id}_tiles：存放瓦片图像
          * {sample_id}_expr：存放对应的表达量数据
      - 每个 _tiles 文件夹中，图像文件命名规则为：
          <sample_id>_<col>_<row>.<ext>
        与 _expr 文件夹中的表达量文件名一致（仅扩展名不同）。

    参数:
      - root_dir: 数据根目录
      - transform: 图像预处理（默认为调整到 (512,512) 并归一化到 [-1,1]）
      - image_exts: 图像文件的扩展名列表
      - expr_ext: 表达量数据的扩展名（默认 '.pt'）
    """
    def __init__(self, root_dir, 
                 transform=None, 
                 image_exts=['.png', '.jpg', '.jpeg'], 
                 expr_ext='.pt'):
        self.root_dir = root_dir
        self.image_exts = image_exts
        self.expr_ext = expr_ext

        # 默认图像预处理：调整尺寸、转 tensor 以及归一化
        if transform is None:
            self.transform = transforms.Compose([
                transforms.Resize((512, 512), interpolation=Image.LANCZOS),
                transforms.ToTensor(),  # [0,1]
                transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])  # 映射到 [-1,1]
            ])
        else:
            self.transform = transform

        # 1. 扫描 root_dir 下所有子文件夹，将 _tiles 和 _expr 文件夹按照 sample_id 配对
        all_subdirs = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.sample_pairs = {}  # key: sample_id, value: dict {'tiles': path, 'expr': path}
        for d in tqdm(all_subdirs, desc="扫描子文件夹"):
            full_path = os.path.join(root_dir, d)
            if d.endswith('_tiles'):
                sample_id = d[:-len('_tiles')]
                self.sample_pairs.setdefault(sample_id, {})['tiles'] = full_path
            elif d.endswith('_expr'):
                sample_id = d[:-len('_expr')]
                self.sample_pairs.setdefault(sample_id, {})['expr'] = full_path

        # 仅保留同时存在 _tiles 和 _expr 的样本
        self.sample_pairs = {sid: paths for sid, paths in self.sample_pairs.items() 
                             if 'tiles' in paths and 'expr' in paths}
        logging.info(f"共找到 {len(self.sample_pairs)} 个样本配对。")

        # 2. 遍历每个样本的 _tiles 文件夹，建立图像与表达量文件的对应关系，同时解析坐标信息
        self.file_pairs = []  # 每个元素为 {'sample_id': ..., 'image_path': ..., 'expr_path': ..., 'col': ..., 'row': ...}
        for sample_id, paths in tqdm(self.sample_pairs.items(), desc="建立文件配对", total=len(self.sample_pairs)):
            tiles_dir = paths['tiles']
            expr_dir = paths['expr']
            # 使用 os.walk 支持子目录结构
            for root_tiles, _, files in os.walk(tiles_dir):
                for file in files:
                    if any(file.lower().endswith(ext) for ext in self.image_exts):
                        image_path = os.path.join(root_tiles, file)
                        base_name = os.path.splitext(file)[0]
                        # 解析文件名，假设最后两个下划线分割的部分分别为 col 和 row，
                        # 如果文件名中含有多个下划线，sample_id 部分可能包含下划线
                        parts = base_name.split('_')
                        if len(parts) >= 3:
                            try:
                                # 取最后两个部分作为坐标，其余部分作为 sample_id_from_file
                                col = int(parts[-2])
                                row = int(parts[-1])
                                sample_id_from_file = "_".join(parts[:-2])
                            except ValueError:
                                col, row = None, None
                                sample_id_from_file = base_name
                        else:
                            col, row = None, None
                            sample_id_from_file = base_name

                        # 构造表达量文件路径（假设命名完全一致，仅扩展名不同）
                        expr_file = base_name + self.expr_ext
                        expr_path = os.path.join(expr_dir, expr_file)
                        if os.path.exists(expr_path):
                            self.file_pairs.append({
                                'sample_id': sample_id,  # 来自文件夹配对
                                'image_path': image_path,
                                'expr_path': expr_path,
                                'col': col,
                                'row': row
                            })
                        else:
                            logging.warning(f"在样本 {sample_id} 中找不到与 {image_path} 对应的表达量文件 {expr_path}。")
        if not self.file_pairs:
            raise ValueError("未找到任何有效的图像与表达量配对，请检查目录结构和文件命名规则。")
        logging.info(f"共找到 {len(self.file_pairs)} 个有效文件配对。")

    def __len__(self):
        return len(self.file_pairs)
    
    def __getitem__(self, idx):
        pair = self.file_pairs[idx]
        sample_id = pair['sample_id']
        image_path = pair['image_path']
        expr_path = pair['expr_path']
        col = pair['col']
        row = pair['row']
        
        # 加载图像
        try:
            image = Image.open(image_path).convert("RGB")
        except Exception as e:
            logging.error(f"加载图像失败：{image_path}，错误信息：{e}")
            raise e
        if self.transform:
            image = self.transform(image)
        
        # 加载表达量数据
        try:
            expr_data = torch.load(expr_path)
            if torch.is_tensor(expr_data) and expr_data.is_sparse:
                expr_data = expr_data.to_dense()
        except Exception as e:
            logging.error(f"加载表达量数据失败：{expr_path}，错误信息：{e}")
            raise e
        
        return {
            "sample_id": sample_id, 
            "image": image, 
            "expression": expr_data,
            "col": col,
            "row": row
        }


In [34]:
root_dir = "/cwStorage/nodecw_group/jijh/hest_output"

dataset = MultiSampleTileExprDataset(root_dir)
logging.info(f"数据集中共有 {len(dataset)} 个文件配对。")

In [36]:
dataset.__getitem__(0)

In [37]:
from src.preprocessing.data_process import construct_affinity_matrix

In [61]:
import os
import torch
import numpy as np
import pandas as pd
import anndata
import scanpy as sc
from scipy.sparse import csr_matrix, diags
from torch_geometric.data import Data
import logging
from tqdm import tqdm


# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)


def extract_sample_data(dataset, sample_id):
    """
    从 MultiSampleTileExprDataset 中提取指定 sample_id 的所有数据。
    
    返回：
      - expr_list: list，每个元素为一个 tile 的表达量数据（numpy 数组）
      - coords_list: list，每个元素为 [x, y] 坐标（x 对应 col，y 对应 row）
      - image_paths: list，每个元素为对应 tile 的图像文件路径（仅作为指针）
      - sample_ids: list，每个元素为 tile 对应的 sample_id
    """
    logging.info(f"开始提取 sample_id 为 {sample_id} 的数据。")
    sample_pairs = [pair for pair in dataset.file_pairs if pair['sample_id'] == sample_id]
    if len(sample_pairs) == 0:
        raise ValueError(f"在数据集中未找到 sample_id 为 {sample_id} 的数据。")
    
    expr_list = []
    coords_list = []
    image_paths = []
    sample_ids = []
    
    for pair in sample_pairs:
        # 加载表达量数据，并处理稀疏 tensor
        expr_data = torch.load(pair['expr_path'])
        if isinstance(expr_data, torch.Tensor):
            if expr_data.is_sparse:
                logging.debug(f"表达量数据 {pair['expr_path']} 为稀疏 tensor，转换为密集格式。")
                expr_data = expr_data.to_dense()
            expr_data = expr_data.cpu().numpy()
        expr_list.append(expr_data)
        
        # 解析坐标（col 作为 x, row 作为 y）
        x, y = pair['col'], pair['row']
        coords_list.append([x, y])
        
        # 保存图像路径作为图像数据指针
        image_paths.append(pair['image_path'])
        sample_ids.append(pair['sample_id'])
    
    logging.info(f"共提取到 {len(expr_list)} 个 tile 数据。")
    return expr_list, coords_list, image_paths, sample_ids


def build_anndata(expr_list, coords_list, sample_ids, target_gene_num=2000):
    """
    根据提取的表达数据、坐标和 sample_id 构建 AnnData 对象。
    如果实际基因数少于 target_gene_num，则补零扩展至 target_gene_num。
    
    返回：
      - adata: 构造好的 AnnData 对象，其中 obs 中保存 sample_id 和坐标，
               obsm['spatial'] 中保存空间坐标。
    """
    logging.info("开始构建 AnnData 对象。")
    # 构造表达矩阵
    X = np.vstack(expr_list)  # 形状 (N, n_genes)
    coords = np.array(coords_list)  # 形状 (N, 2)
    n_samples, n_genes = X.shape
    logging.info(f"初始表达矩阵维度：{n_samples} 个样本，{n_genes} 个基因。")
    
    # 若基因数不足 target_gene_num，则补零扩展
    if n_genes < target_gene_num:
        pad_width = target_gene_num - n_genes
        logging.info(f"基因数不足 {target_gene_num}，补零扩展 {pad_width} 个基因。")
        X = np.hstack([X, np.zeros((n_samples, pad_width))])
        gene_names = [f"gene_{i}" for i in range(n_genes)] + [f"pad_{i}" for i in range(pad_width)]
    else:
        gene_names = [f"gene_{i}" for i in range(n_genes)]
    
    # 构造 AnnData 对象
    adata = anndata.AnnData(X)
    adata.var_names = gene_names
    adata.obs['sample_id'] = sample_ids
    adata.obs['x'] = coords[:, 0]
    adata.obs['y'] = coords[:, 1]
    adata.obsm['spatial'] = coords
    
    logging.info(f"AnnData 对象构建完成，形状为 {adata.shape}。")
    return adata


def add_qc_metrics(adata, mt_prefix='MT-'):
    """
    计算质检指标（如总表达量、检测到的基因数等）。
    如有需要，也可以在 adata.var 中标记线粒体基因，然后在 calculate_qc_metrics 中使用。
    
    参数：
      - adata: AnnData 对象
      - mt_prefix: 用于标记线粒体基因的前缀（可选）
    """
    logging.info("开始计算质检指标。")
    # 如果有线粒体基因信息，可以将标记写入 adata.var
    adata.var['mt'] = [gene.startswith(mt_prefix) for gene in adata.var_names]
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], inplace=True)
    logging.info("质检指标计算完成。")


def preprocess_anndata(adata, target_gene_num=2000, pca_n_comps=50):
    """
    对 AnnData 对象进行预处理，包括归一化、对数转换、高变基因选择、数据缩放和 PCA 降维。
    
    如果高变基因数量不足 target_gene_num，则补零扩展。
    
    返回：
      - 处理后的 AnnData 对象
    """
    logging.info("开始预处理 AnnData 对象：归一化、对数转换、高变基因选择、缩放及 PCA。")
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    
    # 计算高变基因并选择前 target_gene_num 个
    sc.pp.highly_variable_genes(adata, n_top_genes=target_gene_num, flavor='seurat', inplace=True)
    if adata.shape[1] < target_gene_num:
        current_genes = adata.shape[1]
        pad_width = target_gene_num - current_genes
        logging.info(f"高变基因数量不足 {target_gene_num}，补零 {pad_width} 列。")
        pad_data = np.zeros((adata.n_obs, pad_width))
        if hasattr(adata.X, "toarray"):
            X_dense = adata.X.toarray()
        else:
            X_dense = adata.X
        X_new = np.hstack([X_dense, pad_data])
        adata.X = X_new
        pad_gene_names = [f"pad_hvg_{i}" for i in range(pad_width)]
        adata.var = adata.var.append(pd.DataFrame(index=pad_gene_names))
    else:
        adata = adata[:, adata.var.highly_variable]
    
    sc.pp.scale(adata, max_value=10)
    sc.tl.pca(adata, n_comps=pca_n_comps, svd_solver='arpack')
    logging.info(f"PCA 完成，PCA 结果维度为 {adata.obsm['X_pca'].shape}。")
    
    return adata


def build_graph_data(adata, coords, sample_ids, image_paths,
                     affinity_mode='radius', cutoff=1.0, n_neighbors=5,
                     metric='euclidean', add_self_loop=False, pca_key='X_pca'):
    """
    根据 AnnData 对象和空间坐标构建 torch_geometric 图数据。
    
    参数：
      - adata: 经过预处理且包含 PCA 结果的 AnnData 对象，节点特征保存在 adata.obsm[pca_key]
      - coords: 坐标数组，形状 (N, 2)
      - sample_ids, image_paths: 用于记录节点的 sample_id 和图像路径
      - affinity_mode, cutoff, n_neighbors, metric, add_self_loop: 构建 affinity matrix 的参数
      
    返回：
      - graph_data: torch_geometric.data.Data 对象，包含节点特征、边信息和附加信息
    """
    logging.info("开始构建图数据。")
    # 构建邻接矩阵，使用外部导入的 construct_affinity_matrix
    affinity_matrix = construct_affinity_matrix(
        coordinates=coords,
        mode=affinity_mode,
        cutoff=cutoff,
        n_neighbors=n_neighbors,
        metric=metric,
        add_self_loop=add_self_loop
    )
    logging.info(f"构建的 affinity matrix 非零元素数：{affinity_matrix.nnz}。")
    
    # 提取 PCA 结果作为节点特征（注意调用 .copy() 避免负步长问题）
    node_features = torch.tensor(adata.obsm[pca_key].copy(), dtype=torch.float)
    
    # 将 affinity_matrix 转换为 COO 格式，提取边索引和边权重
    affinity_coo = affinity_matrix.tocoo()
    edge_index = torch.tensor([affinity_coo.row, affinity_coo.col], dtype=torch.long)
    edge_weight = torch.tensor(affinity_coo.data, dtype=torch.float)
    
    pos = torch.tensor(coords, dtype=torch.float)  # 节点空间坐标
    
    # 构造 torch_geometric 图数据对象
    graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_weight, pos=pos)
    graph_data.sample_ids = sample_ids
    graph_data.image_paths = image_paths
    
    logging.info(f"图数据构建完成，节点数：{node_features.shape[0]}，边数：{edge_index.shape[1]}。")
    return graph_data


def create_anndata_and_graph(dataset, sample_id,
                             target_gene_num=2000,
                             pca_n_comps=50,
                             affinity_mode='radius',
                             cutoff=1.0,
                             n_neighbors=5,
                             metric='euclidean',
                             add_self_loop=False,
                             pca_key='X_pca'):
    """
    综合调用各个模块函数，根据指定 sample_id 从 dataset 中构建 AnnData 对象和 torch_geometric 图数据。
    
    返回：
      - adata: 包含表达数据、质检指标、预处理和 PCA 结果的 AnnData 对象
      - graph_data: 包含节点特征、边信息和附加信息（如图像路径、sample_id）的图数据对象
    """
    logging.info(f"开始创建 sample_id {sample_id} 的 AnnData 和图数据。")
    # 1. 数据提取
    expr_list, coords_list, image_paths, sample_ids = extract_sample_data(dataset, sample_id)
    coords = np.array(coords_list)
    
    # 2. 构建 AnnData
    adata = build_anndata(expr_list, coords_list, sample_ids, target_gene_num=target_gene_num)
    
    # 3. 质检计算
    add_qc_metrics(adata, mt_prefix='MT-')
    
    # 4. 预处理及降维（PCA）
    adata = preprocess_anndata(adata, target_gene_num=target_gene_num, pca_n_comps=pca_n_comps)
    
    # 5. 构建图数据
    graph_data = build_graph_data(adata, coords, sample_ids, image_paths,
                                  affinity_mode=affinity_mode, cutoff=cutoff,
                                  n_neighbors=n_neighbors, metric=metric,
                                  add_self_loop=add_self_loop, pca_key=pca_key)
    
    logging.info("AnnData 与图数据创建完成。")
    return adata, graph_data




In [67]:
# =================== 示例代码 ===================
# 请确保 dataset 已经正确构造，例如：
# from your_dataset_module import MultiSampleTileExprDataset
# root_dir = "/your/data/root_dir"
# dataset = MultiSampleTileExprDataset(root_dir)
    
target_sample_id = "NCBI371"  # 根据实际情况替换

adata, graph_data = create_anndata_and_graph(
    dataset=dataset,
    sample_id=target_sample_id,
    target_gene_num=2000,
    pca_n_comps=50,
    affinity_mode='number',  # 可选 'radius' 或 'number'
    cutoff=None,             # mode='number' 时可以设为 None
    n_neighbors=8,
    metric='euclidean',
    add_self_loop=True,
    pca_key='X_pca'
)
logging.info("流程运行成功！")
print(adata)
print(graph_data)


In [68]:
adata

In [69]:
graph_data

In [70]:
graph_data.x

In [71]:
# 配置日志记录
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)

# 输出目录
output_dir = "/cwStorage/nodecw_group/jijh/hest_graph_data"
os.makedirs(output_dir, exist_ok=True)
logging.info(f"输出目录设置为: {output_dir}")


In [72]:
# 获取所有的 sample_id（这里基于 dataset.sample_pairs 的 key）
all_sample_ids = list(dataset.sample_pairs.keys())
logging.info(f"将处理 {len(all_sample_ids)} 个 sample_id。")

# 遍历每个 sample_id，生成图数据并保存
for sid in all_sample_ids:
    logging.info(f"开始处理 sample_id: {sid}")
    try:
        # 创建 AnnData 和图数据
        adata, graph_data = create_anndata_and_graph(
            dataset=dataset,
            sample_id=sid,
            target_gene_num=2000,
            pca_n_comps=50,
            affinity_mode='number',  # 固定邻居数模式
            cutoff=None,             # number 模式下 cutoff 设为 None
            n_neighbors=8,           # 邻居数设置为 8
            metric='euclidean',
            add_self_loop=True,
            pca_key='X_pca'
        )
        # 保存生成的图数据（这里以 .pt 格式保存）
        output_file = os.path.join(output_dir, f"{sid}_graph.pt")
        torch.save(graph_data, output_file)
        logging.info(f"成功保存 sample_id {sid} 的图数据到: {output_file}")
    except Exception as e:
        logging.error(f"处理 sample_id {sid} 时发生错误: {e}")